# Properties

In [1]:
import iso6346

In [2]:
class ContenedorDeMercancias:
    
    # Definición de atributos de clase
    
    siguiente_codigo_de_serie = 1337
    
    @staticmethod
    def _crear_codigo_BIC(codigo_propietario, codigo_de_serie):
        return iso6346.create(
        owner_code=codigo_propietario,
        serial=str(codigo_de_serie).zfill(6))
    
    @classmethod
    def _generar_codigo_de_serie(cls):
        
        resultado = cls.siguiente_codigo_de_serie 
        cls.siguiente_codigo_de_serie += 1
        return resultado
    
    # Crea un contenedor vacío
    @classmethod
    def crear_contenedor_vacio(cls, codigo_propietario, **kwargs):
        return cls(codigo_propietario, contenidos = [], **kwargs)
    
    # Crea un contenedor a partir de un codigo de propietario y unos items
    @classmethod
    def crear_contenedor_con_elementos(cls, codigo_propietario, contenidos, **kwargs):
        return cls(codigo_propietario, list(contenidos), **kwargs)
        
    
    def __init__(self, codigo_propietario, contenidos, **kwargs):
        
        # Definición de atributos de instancia
        
        self.codigo_propietario = codigo_propietario
        self.contenidos = contenidos
        self.bic = ContenedorDeMercancias._crear_codigo_BIC(
            codigo_propietario=codigo_propietario,
            codigo_de_serie=ContenedorDeMercancias._generar_codigo_de_serie()
        )

In [3]:
class ContenedorDeMercanciasRefrigerado(ContenedorDeMercancias):
    
    MAX_CELSIUS = 4.0
        
    def __init__(self, codigo_propietario, contenidos, *, celsius, **kwargs):
        super().__init__(codigo_propietario, contenidos, **kwargs)
        
        if celsius > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
            raise ValueError("La temperatura es demasiado alta")
        
        self.celsius = celsius
    
    @staticmethod
    def _crear_codigo_bic(codigo_propietario, codigo_de_serie):
        return iso6346.create(
        owner_code=codigo_propietario,
        serial=str(codigo_de_serie).zfill(6),
        category="R")

Como hemos visto en el final del notebook anterior, una vez que es creada la instancia de un contenedor refrigerado, se puede cambiar su temperatura modificando el atributo celsius:

In [4]:
r3 = ContenedorDeMercanciasRefrigerado.crear_contenedor_con_elementos("YML", ["fish"], celsius=2.0)

In [5]:
r3

In [6]:
r3.celsius

2.0

In [7]:
r3.celsius = 10

In [8]:
r3.celsius

10

Esto no está bien y no debería ser posible.

Podemos arreglar esto con la forma que tiene Python de encapsular los getters y los setters: **properties**.

Para ello vamos a hacer lo siguiente:

- **Renombrar el atributo celsius a _celsius**, para indicar que es de visibilidad privada, es decir, que no se debe acceder a el directamente.
- **Crear el método celsius con el decorador @property**, el cual simula el getter de la propiedad **_celsius**
- **Crera el método celsius con el decorador @celsius.setter**, el cual simula el setter de la propiedad **_celsius**

In [9]:
class ContenedorDeMercanciasRefrigerado(ContenedorDeMercancias):
    
    MAX_CELSIUS = 4.0
        
    def __init__(self, codigo_propietario, contenidos, *, celsius, **kwargs):
        super().__init__(codigo_propietario, contenidos, **kwargs)
        
        if celsius > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
            raise ValueError("La temperatura es demasiado alta")
        
        # Renombramos celsius a _celsius
        
        self._celsius = celsius
        
    @property
    def celsius(self):
        return self._celsius
    
    @celsius.setter
    def celsius(self, value):
        
        if value > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
            raise ValueError("La temperatura es demasiado alta")
            
        self._celsius = value
    
    @staticmethod
    def _crear_codigo_bic(codigo_propietario, codigo_de_serie):
        return iso6346.create(
        owner_code=codigo_propietario,
        serial=str(codigo_de_serie).zfill(6),
        category="R")

In [10]:
r3 = ContenedorDeMercanciasRefrigerado.crear_contenedor_con_elementos("YML", ["fish"], celsius=2.0)

In [11]:
r3

Ahora podemos acceder a _celsius como si no fuera un atributo privado:

In [12]:
r3.celsius

2.0

Y podemos asignarle un valor como si no fuera un atributo privado, manteniendo las correspondientes restricciones:

In [13]:
try:
    r3.celsius = 10
except ValueError as error:
    print(error)

La temperatura es demasiado alta


In [14]:
r3.celsius = 1.0
r3.celsius

1.0

Ahora vamos a añadir soporte para grados farenheit:

In [15]:
class ContenedorDeMercanciasRefrigerado(ContenedorDeMercancias):
    
    MAX_CELSIUS = 4.0
        
    def __init__(self, codigo_propietario, contenidos, *, celsius, **kwargs):
        super().__init__(codigo_propietario, contenidos, **kwargs)
        
        if celsius > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
            raise ValueError("La temperatura es demasiado alta")
        
        # Renombramos _celsius a celsius, autoencapsulándolo y accediendo a través de su property
        
        self.celsius = celsius
        
    # Creamos estos dos métodos para pasar de celsius a farenheit y viceversa
    
    @staticmethod
    def _celsius_to_farenheit(celsius):
        return celsius * 9/5 + 32
    
    @staticmethod
    def _farenheit_to_celsius(farenheit):
        return (farenheit -32) * 5/9 
    
    @property
    def celsius(self):
        return self._celsius
    
    @celsius.setter
    def celsius(self, value):
        
        if value > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
            raise ValueError("La temperatura es demasiado alta")
            
        self._celsius = value
        
    # Creamos los getter y setter de farenheit
    
    @property
    def farenheit(self):
        return ContenedorDeMercanciasRefrigerado._celsius_to_farenheit(self.celsius)
    
    @farenheit.setter
    def farenheit(self, value):
        self.celsius = ContenedorDeMercanciasRefrigerado._farenheit_to_celsius(value)
        
    
    
    @staticmethod
    def _crear_codigo_bic(codigo_propietario, codigo_de_serie):
        return iso6346.create(
        owner_code=codigo_propietario,
        serial=str(codigo_de_serie).zfill(6),
        category="R")

Como podemos ver, hemos añadido dos métodos para pasar de celsius a farenheit y viceversa.

Además hemos añadido un getter y setter para farenheit que calcula sobre la marcha el valor de los grados farenheit apoyándose en el atributo _celsius.

De esta forma no será necesario definir un atributo _farenheit.

Por último, hemos autoencapsulado el atributo celsius, pasando de **self._celsius** a **self.celsius**, asignando su valor a través de su propia property en vez de directamente.

In [16]:
r5 = ContenedorDeMercanciasRefrigerado.crear_contenedor_vacio("GOG", celsius=-20.0)

In [17]:
r5.farenheit

-4.0

In [18]:
r5.celsius

-20.0

In [19]:
try:
    r5.farenheit = 100
except ValueError as error:
    print(error)

La temperatura es demasiado alta


In [20]:
try:
    r5.farenheit = 10
except ValueError as error:
    print(error)

In [21]:
r5.farenheit

10.0

In [22]:
r5.celsius

-12.222222222222221